<a href="https://colab.research.google.com/github/amishgog/exoplanet-search/blob/main/create_train_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#uploading star ID file for light curve extraction
from google.colab import files
uploaded = files.upload()
filename = next(iter(uploaded))

In [ ]:
#importing necessary libraries and reading csv
import warnings
warnings.filterwarnings("ignore")
import sys
import csv
import pandas as pd
import io
data = pd.read_csv(filename)
print(data)

!pip install lightkurve
#installing tsfresh for feature extraction
!pip install tsfresh

In [ ]:
#main file for extracting features from light curves and storing in csv
from lightkurve import search_targetpixelfile
from tsfresh import extract_features
from tsfresh.feature_extraction import extract_features, MinimalFCParameters
from csv import writer
appended_data = []
with open(filename) as file:
    reader = csv.DictReader(file)

    for row in reader:
      try:
          tpf_file = search_targetpixelfile(row['\ufeffID']).download(quality_bitmask='default')
          if tpf_file == None:
            continue
          lc = tpf_file.to_lightcurve()
          lc.to_csv("light_curve.csv", overwrite = True)
          df = pd.read_csv("light_curve.csv")
          df["ID"] = 1
          extracted_features = extract_features(df, default_fc_parameters=MinimalFCParameters(), column_id = "ID")
          extracted_features['Boolean'] = row['Yes/No']
          extracted_features.insert(loc = 0, column = "Star_ID", value = row['\ufeffID'])
          appended_data.append(extracted_features)
          
     
      except (TypeError , AttributeError, ValueError):
          continue
    


In [ ]:
#appending data to a single file 
appended_data = pd.concat(appended_data)
appended_data.to_excel("final_train_file.xlsx")
#converting file to an excel to feed into ML algorithm
with open("final_train_file.xlsx" , mode = 'rb') as the_file:
  df1 = pd.read_excel(the_file)
  data_frame = df1.drop(['quality__median' , 'quality__minimum'], axis=1)
  display(data_frame)
  